In [1]:
import os
import sys
sys.path.append("../")

### Convert to assistant-request to get the assistant for a call

In [6]:
from app.database.session import db_session
db=db_session()


In [7]:
from app.database.tables.phone_number import PhoneNumber
import requests
from app.core.config import settings

In [8]:
phone_numbers = db.query(PhoneNumber).all()

In [21]:
for phone_number in phone_numbers:
    if phone_number.vapi_phone_number_id is None:
        print(f"Ignoring {phone_number.pn}")
        continue
    res = requests.patch(f"https://api.vapi.ai/phone-number/{phone_number.vapi_phone_number_id}", headers={"Authorization": f"Bearer {settings.VAPI_API_KEY}"}, json={"serverUrl": "https://develop-api.syntag.ai/vapi/server-url", "serverUrlSecret": settings.VAPI_SECRET, "assistantId": None})
    print(res.status_code)
    if res.status_code >=400:
        print(phone_number.uuid, phone_number.vapi_phone_number_id)
        print( res.json())
        break
    print(res.json())


Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
200
{'id': '55015b2c-33f9-4661-b106-61ebe86fd324', 'orgId': '27973b53-5b0b-4d0c-967b-1a7fe636b48a', 'number': '+12294148610', 'createdAt': '2024-07-20T21:32:54.444Z', 'updatedAt': '2024-09-15T05:05:13.182Z', 'twilioAccountSid': 'AC01b78261b4915ee8edba6b7492c3c71f', 'twilioAuthToken': '3f2bfadd485fbbdbdcfd94dba4979035', 'serverUrl': 'https://develop-api.syntag.ai/vapi/server-url', 'serverUrlSecret': 'hxMj4Ywu2whEp1Mm2CW1fln3kX25dHkk3wTQbVPvzFo=', 'provider': 'twilio'}
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
Ignoring +14155552671
200
{'id': 'a1292477-2f2e-4974-9cdb-326e5ed83b86', 'orgId': '27973b53-5b0b-4d0c-967b-1a7fe636b48a', 'number': '+12517582074', 'createdAt': '2024-08-22T17:48:51.874Z', 'updatedAt': '2024-09-15T05:05:13.777Z', 'twilioAccountSid': 'AC01b78261b49

In [3]:


res = requests.get("https://api.vapi.ai/phone-number?limit=1000", headers={"Authorization": f"Bearer {settings.VAPI_API_KEY}"})

In [5]:
res.json()

[{'id': '0d34118a-e570-4f39-9949-224a7071637f',
  'orgId': '27973b53-5b0b-4d0c-967b-1a7fe636b48a',
  'number': '+14122845412',
  'createdAt': '2024-09-14T23:28:57.173Z',
  'updatedAt': '2024-09-14T23:28:57.173Z',
  'twilioAccountSid': 'AC01b78261b4915ee8edba6b7492c3c71f',
  'twilioAuthToken': '3f2bfadd485fbbdbdcfd94dba4979035',
  'provider': 'twilio'},
 {'id': 'a9676700-0951-4baa-a5c9-7f563ed4147e',
  'orgId': '27973b53-5b0b-4d0c-967b-1a7fe636b48a',
  'assistantId': '4e9a0b6c-7ccf-4b4c-bd9d-850cb76e8c81',
  'number': '+13018898191',
  'createdAt': '2024-09-14T23:26:18.285Z',
  'updatedAt': '2024-09-14T23:26:19.029Z',
  'twilioAccountSid': 'AC01b78261b4915ee8edba6b7492c3c71f',
  'twilioAuthToken': '3f2bfadd485fbbdbdcfd94dba4979035',
  'provider': 'twilio'},
 {'id': '3c66319b-5a1d-4085-8092-a0f6ad0286ec',
  'orgId': '27973b53-5b0b-4d0c-967b-1a7fe636b48a',
  'assistantId': 'ca60b8b0-0ac8-47ec-b404-459770ecc55e',
  'number': '+13014507491',
  'createdAt': '2024-09-13T16:06:39.690Z',
  'upd

### Analyze call costs

In [40]:
from app.core.config import settings
import requests
res = requests.get("https://api.vapi.ai/call?limit=1000", headers={"Authorization": f"Bearer {settings.VAPI_API_KEY}"})


In [41]:
res_json = res.json()

In [42]:
len(res_json)

1000

In [62]:
from datetime import datetime
from dateutil import parser
points = []
for i, call in enumerate(res_json):
    try:
        date1 = parser.isoparse(call["startedAt"])
        date2 = parser.isoparse(call["endedAt"])

        time_difference = (date2 - date1).total_seconds()
        if time_difference < 1:
            points.append(0)
            continue

        cost = call["cost"]

        cost_per_minute = (cost / time_difference ) * 60
        points.append(cost_per_minute)
        print(i, cost_per_minute)
    except Exception as e:
        points.append(0)
        pass







0 0.06883185528865918
1 0.07674684994272622
2 0.07211444248481286
3 0.06964813183396383
4 0.07319562575941677
5 0.07522446008250426
6 0.06714296688945226
7 0.06899498024137563
8 0.07776261937244201
9 0.0772935227546352
10 0.06938656846593662
11 0.07134071340713406
12 0.06688593738230937
13 0.06519268809360139
14 0.07579093753621509
15 0.0706454697916811
17 0.07043682425577552
18 0.07994825355756792
20 0.09267879813796022
21 0.0685705674026072
22 0.07127830714020542
23 0.06557881773399014
24 0.07295358285072176
25 0.07026327632763275
26 0.07157663183721816
27 0.0671393751384888
28 0.06631365720174846
29 0.06629209246200837
30 0.07584167240610946
31 0.06432641058628928
32 0.0734130634774609
33 0.06851295229791432
34 0.06631178707224333
35 0.07707023776988248
36 0.068821267120977
37 0.06825975722606281
38 0.06513730935631577
39 0.06649153002989402
40 0.0637331256490135
41 0.06456087730808885
42 0.06432038834951456
43 0.06689965052421368
44 0.0712661106899166
45 0.06469663627790002
46 0.06

In [66]:
res_json[264]

{'id': '52e580f7-c184-4105-8dcd-4e5a725b06ec',
 'phoneNumberId': '55015b2c-33f9-4661-b106-61ebe86fd324',
 'type': 'inboundPhoneCall',
 'startedAt': '2024-07-24T01:11:21.629Z',
 'endedAt': '2024-07-24T01:11:27.271Z',
 'transcript': "AI: Hello, I'm Clara.  can I help you today?\nUser: What is tennis?\n",
 'recordingUrl': 'https://auth.vapi.ai/storage/v1/object/public/recordings/52e580f7-c184-4105-8dcd-4e5a725b06ec-1721783489785-caa93891-35fb-4827-aa7b-352075f8e61c-mono.wav',
 'summary': "Tennis is a racket sport played between two players or two teams of two players each. It involves hitting a ball over a net on a rectangular court, with the goal of making it difficult for the opponent to return the ball within the court boundaries.\n\nAI: Tennis is a popular racket sport played on a rectangular court. Players use rackets to hit a ball back and forth over a net, aiming to score points by making it difficult for their opponent to return the ball within the court's boundaries. It can be pl

In [70]:
import numpy as np

print(np.percentile(points, 10))
print(np.percentile(points, 25))
print(np.percentile(points, 50))
print(np.percentile(points, 75))
print(np.percentile(points, 90))
print(np.percentile(points, 95))
print(np.percentile(points, 99))

0.07322510444670464
0.05977165375216332
0.06488241632002502
0.071272208915061
0.08169622363190168
0.09811603337264597
0.1118311008417132
0.1894973121693401


In [65]:
np.max(points)

0.7199574618929457